In [ ]:
import sys, cv2, numpy, time
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5 import QtGui
from Video import *
from Prediction import *
from difflib import SequenceMatcher

class LipService(QWidget):
    str_list =['오와우애오','임애','임애애오','임애오','우우','우우애오','안어아애오','안어','안어이아애오','안어이애애오','오임이아애오','오임이아','외오','외오암이아','외오애오','암아','암아암이아','암아애오','오아','오아오','오아암이아','이어','이어오','이어암이아']
    answer_list = ['도와주세요','힘내','힘내세요', '힘내요','누구', '누구세요', '안녕하세요', '안녕','안녕히가세요','안녕히계세요','조심히가세요','조심히가','죄송','죄송합니다','죄송해요','감사','감사합니다','감사해요','좋아','좋아요','좋아합니다','싫어','싫어요','싫어합니다']
    
    def __init__(self):
        super().__init__()
        #윈도우 설정
        self.setWindowTitle('LipService')
        self.center() # self.move(500,200)    # 창 위치
        self.setWindowFlags(Qt.WindowStaysOnTopHint)
        self.resize(1000, 600)   # 창 크기
        self.idx = 0
        self.words = ""
        self.language = ""
        self.result = "입을 크게 벌리고 또박또박 말해주세요."
        self.initUI()
        self.video = Video(self, QSize(self.frm.width(), self.frm.height()))
        

    def initUI(self):
        layout = QVBoxLayout()
        '''
        grid = QGridLayout()
        grid.addWidget(self.languageCheckboxes(), 0, 1)
        grid.addWidget(self.ButtonGroup(), 1, 1)
        self.setLayout(grid)
        
        '''
        # 웹캠 area
        self.frm = QLabel(self)
        self.frm.setMinimumSize(QSize(640,480))
        self.frm.setFrameShape(QFrame.Panel)
        
        #언어 체크박스
        self.ecb = QCheckBox('영어', self)
        self.ecb.setChecked(True)
        self.ecb.setFont(QtGui.QFont('Hack', 15))
        self.ecb.move(800, 100)
        
        self.ecb.toggle()
        self.ecb.stateChanged.connect(self.changeLanguage)
        layout.addWidget(self.ecb)
        
        self.kcb = QCheckBox('한국어', self)
        self.kcb.setChecked(True)
        self.kcb.setFont(QtGui.QFont('Hack', 15))
        self.kcb.move(800, 150)
        
        self.kcb.toggle()
        self.kcb.stateChanged.connect(self.changeLanguage)
        layout.addWidget(self.kcb)
        
        #체크박스 하나만 선택
        buttongroup = QButtonGroup(self)
        buttongroup.setExclusive(True)
        buttongroup.addButton(self.ecb, 1)
        buttongroup.addButton(self.kcb, 2)
        
        # 위치 조정을 위한 여백
        self.label = QLabel(self)
        self.label.setAlignment(Qt.AlignCenter)
        self.label.resize(600, 200)
        self.label.move(200,450)
        self.font = self.label.font()
        self.font.setPointSize(30)
        self.font.setBold(True)
        self.label.setFont(self.font)
        layout.addWidget(self.label)
        
        # 웹캠 start/stop 버튼
        self.sbtn = QPushButton('START CAM',self)
        self.sbtn.setCheckable(True)
        self.sbtn.setChecked(False)
        self.sbtn.move(800, 230)
        self.sbtn.resize(100,50)
        self.sbtn.clicked.connect(self.sbtn_clicked)
        layout.addWidget(self.sbtn)
        
        # predict 버튼
        pbtn = QPushButton('PREDICT',self)
        pbtn.move(800, 310) 
        pbtn.resize(100,50)
        pbtn.clicked.connect(self.pbtn_clicked)
        layout.addWidget(pbtn)
        
        # text 레이블
        self.text = QLabel(self.result, self)
        self.text.setAlignment(Qt.AlignCenter)
        self.text.setFrameShape(QFrame.Box)
        fontT = self.text.font()
        fontT.setPointSize(30)
        fontT.setBold(True)
        self.text.setFont(fontT)
        
        # 레이아웃 구성
        hbox = QHBoxLayout()
        hbox.addWidget(self.frm)
        hbox.addLayout(layout)
        mainLayout = QVBoxLayout()
        mainLayout.addLayout(hbox)
        mainLayout.addWidget(self.text)
        self.setLayout(mainLayout)
        
    # 언어 체크박스 선택 시
    def changeLanguage(self):
        if self.ecb.isChecked():
            self.language = "eng"
        if self.kcb.isChecked():
            self.language = "kor"
            
        print(self.language)
    
    #start/stop 버튼 클릭 시
    def sbtn_clicked(self):
        if self.sbtn.isChecked():
            btn = self.sender()
            btn.setText('STOP CAM')
            
            #웹캠 시작
            self.video.startCam()

        else:
            btn = self.sender()
            btn.setText('START CAM')
            
            #웹캠 종료 및 글자 분석
            self.video.stopCam()
            self.prediction = Prediction(self, self.language)
            if self.language == "kor":
                self.data = np.expand_dims(self.video.x_data, axis = 0)
#                 self.data = self.video.x_data[:5].reshape(-1, 5, 100, 200, 3)
                
            elif self.language == "eng":
                self.data = np.expand_dims(self.video.x_data, axis = 0)
#                 self.data = self.video.x_data[:27].reshape(-1, 27, 64, 64, 3)
            
            self.data = np.expand_dims(self.data, axis = 0)
            print("data shape: ", self.data.shape)
            self.prediction.predict(self.data)
                
    # predict 버튼 클릭 시
    def pbtn_clicked(self):
        # "분석 중입니다" 레이블 표시
        self.text.setText("분석 중입니다.")
        if self.language == "eng":
            print("final result: ", self.words)
            self.result = self.words
            self.text.setText(self.result)

        elif self.language == "kor":
            # 클러스터링 함수 실행
            print("str result: ", self.words)
            self.result = self.clusteringKor(self.words)
            print("final result: ", self.result)
            self.text.setText(self.result)
            self.words = ""

        else:
            self.text.setText("분석할 언어를 선택하세요.")
    
    #화면 가운데로 띄우기  
    def center(self):
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())

    # 웹캠으로부터 전송된 이미지 박스에 맞춰 출력하기 
    def recvImage(self, img):
        self.frm.setPixmap(QPixmap.fromImage(img))
        
    # Predict 값 불러오기
    def getResult(self, str):
        # 한국어일 때는 한 단어씩 predict해서 word에 추가
        if self.language == "kor":
            self.words += str
                
        else:
            self.words = str
        
    # 클러스터링
    def clusteringKor(self, str):
        score = []
        for i in range(len(self.str_list)):
            ratio = SequenceMatcher(None, str, self.str_list[i]).ratio()
            score.append(ratio)
        index = score.index(max(score))
        return self.answer_list[index]
    
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = LipService()
    ex.show()
    app.exec_()